In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()
df = spark.read.json('/FileStore/tables/people.json')

In [3]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [4]:
 df.printSchema()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [5]:
df.columns

Out[5]: ['age', 'name']

In [6]:
df.describe().show()

+-------+------------------+-------+
summary| age| name|
+-------+------------------+-------+
 count| 2| 3|
 mean| 24.5| null|
 stddev|7.7781745930520225| null|
 min| 19| Andy|
 max| 30|Michael|
+-------+------------------+-------+

In [7]:
# import type tools
from pyspark.sql.types import (StructField, StringType, IntegerType, StructType)

In [8]:
# Create a list of structure field, with 3 parameters: the column name('age'), the class instances(IntegerType) 
# and some sort of nullable(whether or not the field can be NULL, if not True, will show error for missing values)

# Change type of 'age' from long to integer
# Change type of 'name' to string
data_schema = [StructField('age', IntegerType(), True),
               StructField('name', StringType(), True)]

In [9]:
# Pass in data_schema into StructType
final_struc = StructType(fields = data_schema)

In [10]:
# Read in the json file but clarify the schema to be as the final_struc
df = spark.read.json('/FileStore/tables/people.json', schema=final_struc)

In [11]:
df.printSchema()

root
-- age: integer (nullable = true)
-- name: string (nullable = true)

In [12]:
# Get a column object using df['column']
type(df['age'])

Out[12]: pyspark.sql.column.Column

In [13]:
# Return a dataframe containing a single column
df.select('age').show()

+----+
 age|
+----+
null|
 30|
 19|
+----+

In [14]:
# Check row objects
df.head(2)[0]

Out[14]: Row(age=None, name='Michael')

In [15]:
type(df.head(2)[0])

Out[15]: pyspark.sql.types.Row

In [16]:
# Create new columns
# Pass the name of the new column and the new column
df.withColumn('double_age', df['age']*2).show()

+----+-------+----------+
 age| name|double_age|
+----+-------+----------+
null|Michael| null|
 30| Andy| 60|
 19| Justin| 38|
+----+-------+----------+

In [17]:
# Rename a column
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
my_new_age| name|
+----------+-------+
 null|Michael|
 30| Andy|
 19| Justin|
+----------+-------+

In [18]:
# Register the dataframe as a SQL temporary view
df.createOrReplaceTempView('people')

In [19]:
results = spark.sql('SELECT * FROM people')
results.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [20]:
new_results = spark.sql('SELECT * FROM people WHERE age=30')
new_results.show()

+---+----+
age|name|
+---+----+
 30|Andy|
+---+----+

In [21]:
spark = SparkSession.builder.appName('ops').getOrCreate()
df = spark.read.csv('/FileStore/tables/appl_stock.csv', inferSchema=True, header=True)

In [22]:
df.printSchema()

root
-- Date: timestamp (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [23]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
2010-01-04 00:00:00| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039|
2010-01-05 00:00:00| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002|
2010-01-06 00:00:00| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|
2010-01-07 00:00:00| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265|
2010-01-08 00:00:00| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|
2010-01-11 00:00:00|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758|
2010-01-12 00:00:00|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211|
2010-01-13 00:00:00| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172|
2010-01-14 00:00:00|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657|
2010-01-15 00:00:00|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997|
2010-01-19 00:00:00| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997|
2010-01-20 00:00:00| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644|
2010-01-21 00:00:00| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455|
2010-01-22 00:00:00|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-25 00:00:00|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002|
2010-01-26 00:00:00|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494|
2010-01-27 00:00:00| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002|
2010-01-28 00:00:00| 204.930004| 205.500004| 198.699995| 199.289995|293375600|25.819922000000002|
2010-01-29 00:00:00| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24.883208|
2010-02-01 00:00:00|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
only showing top 20 rows

In [24]:
df.head(3)

Out[24]: [Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date=datetime.datetime(2010, 1, 6, 0, 0), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [25]:
# Use df.filter with sql syntac
df.filter('Close < 500').select(['Open', 'Close']).show()

+------------------+------------------+
 Open| Close|
+------------------+------------------+
 213.429998| 214.009998|
 214.599998| 214.379993|
 214.379993| 210.969995|
 211.75| 210.58|
 210.299994|211.98000499999998|
212.79999700000002|210.11000299999998|
209.18999499999998| 207.720001|
 207.870005| 210.650002|
210.11000299999998| 209.43|
210.92999500000002| 205.93|
 208.330002| 215.039995|
 214.910006| 211.73|
 212.079994| 208.069996|
206.78000600000001| 197.75|
202.51000200000001| 203.070002|
205.95000100000001| 205.940001|
 206.849995| 207.880005|
 204.930004| 199.289995|
 201.079996| 192.060003|
192.36999699999998| 194.729998|
+------------------+------------------+
only showing top 20 rows

In [26]:
df.filter(df['close'] < 500).select('Volume').show()

+---------+
 Volume|
+---------+
123432400|
150476200|
138040000|
119282800|
111902700|
115557400|
148614900|
151473000|
108223500|
148516900|
182501900|
153038200|
152038600|
220441900|
266424900|
466777500|
430642100|
293375600|
311488100|
187469100|
+---------+
only showing top 20 rows

In [27]:
df.filter((df['Close'] < 200) & ~(df['open'] > 200)).show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
2010-02-01 00:00:00|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131|
2010-02-02 00:00:00| 195.909998| 196.319994|193.37999299999998| 195.859997|174585600|25.375532999999997|
2010-02-03 00:00:00| 195.169994| 200.200003| 194.420004| 199.229994|153832000|25.812148999999998|
2010-02-04 00:00:00| 196.730003| 198.370001| 191.570005| 192.050003|189413000| 24.881912|
2010-02-05 00:00:00|192.63000300000002| 196.0| 190.850002| 195.460001|212576700|25.323710000000002|
2010-02-08 00:00:00| 195.690006|197.88000300000002| 193.999994|194.11999699999998|119567700| 25.1501|
2010-02-09 00:00:00| 196.419996| 197.499994| 194.749998|196.19000400000002|158221700| 25.418289|
2010-02-10 00:00:00| 195.889997| 196.6| 194.26|195.12000700000002| 92590400| 25.27966|
2010-02-11 00:00:00| 194.880001| 199.750006|194.05999599999998| 198.669994|137586400| 25.739595|
2010-02-23 00:00:00| 199.999998| 201.330002| 195.709993| 197.059998|143773700| 25.531005|
2014-06-09 00:00:00| 92.699997| 93.879997| 91.75| 93.699997| 75415000| 88.906324|
2014-06-10 00:00:00| 94.730003| 95.050003| 93.57| 94.25| 62777000| 89.428189|
2014-06-11 00:00:00| 94.129997| 94.760002| 93.470001| 93.860001| 45681000| 89.058142|
2014-06-12 00:00:00| 94.040001| 94.120003| 91.900002| 92.290001| 54749000| 87.568463|
2014-06-13 00:00:00| 92.199997| 92.440002| 90.879997| 91.279999| 54525000| 86.610132|
2014-06-16 00:00:00| 91.510002| 92.75| 91.449997| 92.199997| 35561000| 87.483064|
2014-06-17 00:00:00| 92.309998| 92.699997| 91.800003| 92.08000200000001| 29726000| 87.36920699999999|
2014-06-18 00:00:00| 92.269997| 92.290001| 91.349998| 92.18| 33514000| 87.46409|
2014-06-19 00:00:00| 92.290001| 92.300003| 91.339996| 91.860001| 35528000| 87.160461|
2014-06-20 00:00:00| 91.849998| 92.550003| 90.900002| 90.910004|100898000| 86.259066|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
only showing top 20 rows

In [28]:
# Collect the result as a row object
result = df.filter(df['Low'] == 197.16).collect()
result

Out[28]: [Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [29]:
row = result[0]

In [30]:
# Convert row to dict
row.asDict()

Out[30]: {'Date': datetime.datetime(2010, 1, 22, 0, 0),
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [31]:
# Grab a key
row.asDict()['Volume']

Out[31]: 220441900

In [32]:
spark = SparkSession.builder.appName('aggs').getOrCreate()
df = spark.read.csv('/FileStore/tables/sales_info.csv', inferSchema=True, header=True)

In [33]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [34]:
df.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [35]:
df.groupBy('Company')

Out[45]: <pyspark.sql.group.GroupedData at 0x7f4d900b9748>

In [36]:
df.groupBy('Company').mean()

Out[48]: DataFrame[Company: string, avg(Sales): double]

In [37]:
df.groupBy('Company').mean().show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

In [38]:
df.groupBy('Company').count().show()

+-------+-----+
Company|count|
+-------+-----+
 APPL| 4|
 GOOG| 3|
 FB| 2|
 MSFT| 3|
+-------+-----+

In [39]:
df.agg({'Sales':'sum'}).show()

+----------+
sum(Sales)|
+----------+
 4327.0|
+----------+

In [40]:
group_data = df.groupBy('Company')

In [41]:
# A more generalized format but the same as .groupby.agg
group_data.agg({'Sales':'max'}).show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [42]:
# Import function from spark
from pyspark.sql.functions import countDistinct, avg, stddev

In [43]:
df.select(countDistinct('Sales').alias('Count sales')).show()

+-----------+
Count sales|
+-----------+
 11|
+-----------+

In [44]:
df.select(stddev('Sales')).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

In [45]:
from pyspark.sql.functions import format_number

sales_std = df.select(stddev('Sales').alias('std'))
# decimals
sales_std.select(format_number('std', 2)).show()

+---------------------+
format_number(std, 2)|
+---------------------+
 250.09|
+---------------------+

In [46]:
sales_std.select(format_number('std', 2).alias('std')).show()

+------+
 std|
+------+
250.09|
+------+

In [47]:
df.orderBy('Sales').show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [48]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+

In [49]:
spark = SparkSession.builder.appName('dates').getOrCreate()
df = spark.read.csv('/FileStore/tables/appl_stock.csv', inferSchema=True, header=True)

In [50]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
2010-01-04 00:00:00| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039|
2010-01-05 00:00:00| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002|
2010-01-06 00:00:00| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|
2010-01-07 00:00:00| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265|
2010-01-08 00:00:00| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|
2010-01-11 00:00:00|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758|
2010-01-12 00:00:00|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211|
2010-01-13 00:00:00| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172|
2010-01-14 00:00:00|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657|
2010-01-15 00:00:00|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997|
2010-01-19 00:00:00| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997|
2010-01-20 00:00:00| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644|
2010-01-21 00:00:00| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455|
2010-01-22 00:00:00|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-25 00:00:00|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002|
2010-01-26 00:00:00|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494|
2010-01-27 00:00:00| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002|
2010-01-28 00:00:00| 204.930004| 205.500004| 198.699995| 199.289995|293375600|25.819922000000002|
2010-01-29 00:00:00| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24.883208|
2010-02-01 00:00:00|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
only showing top 20 rows

In [51]:
df.select(['Date', 'Open']).show()

+-------------------+------------------+
 Date| Open|
+-------------------+------------------+
2010-01-04 00:00:00| 213.429998|
2010-01-05 00:00:00| 214.599998|
2010-01-06 00:00:00| 214.379993|
2010-01-07 00:00:00| 211.75|
2010-01-08 00:00:00| 210.299994|
2010-01-11 00:00:00|212.79999700000002|
2010-01-12 00:00:00|209.18999499999998|
2010-01-13 00:00:00| 207.870005|
2010-01-14 00:00:00|210.11000299999998|
2010-01-15 00:00:00|210.92999500000002|
2010-01-19 00:00:00| 208.330002|
2010-01-20 00:00:00| 214.910006|
2010-01-21 00:00:00| 212.079994|
2010-01-22 00:00:00|206.78000600000001|
2010-01-25 00:00:00|202.51000200000001|
2010-01-26 00:00:00|205.95000100000001|
2010-01-27 00:00:00| 206.849995|
2010-01-28 00:00:00| 204.930004|
2010-01-29 00:00:00| 201.079996|
2010-02-01 00:00:00|192.36999699999998|
+-------------------+------------------+
only showing top 20 rows

In [52]:
from pyspark.sql.functions import dayofmonth, hour, dayofyear, month, year, weekofyear, format_number, date_format

In [53]:
df.select(dayofmonth(df['Date'])).show()

+----------------+
dayofmonth(Date)|
+----------------+
 4|
 5|
 6|
 7|
 8|
 11|
 12|
 13|
 14|
 15|
 19|
 20|
 21|
 22|
 25|
 26|
 27|
 28|
 29|
 1|
+----------------+
only showing top 20 rows

In [54]:
df.select(hour(df['Date'])).show()

+----------+
hour(Date)|
+----------+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
+----------+
only showing top 20 rows

In [55]:
df.select(month(df['Date'])).show()

+-----------+
month(Date)|
+-----------+
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 2|
+-----------+
only showing top 20 rows

In [56]:
# Create a new Year column
newdf = df.withColumn("Year", year(df['Date']))

In [57]:
newdf.groupBy('Year').mean().show()

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
Year| avg(Open)| avg(High)| avg(Low)| avg(Close)| avg(Volume)| avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547| 5.18378869047619E7|115.96740080555561| 2015.0|
2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143| 1.016087E8| 62.61798788492063| 2013.0|
2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523| 2014.0|
2012| 576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002| 1.319642044E8| 74.81383696800002| 2012.0|
2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60400786904763| 3.84153623015873E7|103.15032854761901| 2016.0|
2010| 259.9576190992064|262.36880881349214|256.84761791269847| 259.8424600000002|1.4982631666666666E8|33.665072424603196| 2010.0|
2011|364.06142773412705| 367.4235704880951|360.29769878174613|364.00432532142867|1.2307474166666667E8| 47.16023692063492| 2011.0|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+

In [58]:
result = newdf.groupBy('Year').mean().select(['Year', 'avg(Close)'])

In [59]:
result.withColumnRenamed('avg(Close)', 'Average Closing Price').show()

+----+---------------------+
Year|Average Closing Price|
+----+---------------------+
2015| 120.03999980555547|
2013| 472.6348802857143|
2014| 295.4023416507935|
2012| 576.0497195640002|
2016| 104.60400786904763|
2010| 259.8424600000002|
2011| 364.00432532142867|
+----+---------------------+

In [60]:
new = result.withColumnRenamed('avg(Close)', 'Average Closing Price')
new.select(['Year', format_number('Average Closing Price', 2).alias('Average Close')]).show()

+----+-------------+
Year|Average Close|
+----+-------------+
2015| 120.04|
2013| 472.63|
2014| 295.40|
2012| 576.05|
2016| 104.60|
2010| 259.84|
2011| 364.00|
+----+-------------+